In [1]:
#test1
import tensorflow

In [23]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:' , x_train.shape)
print('x_test:' , x_test.shape)
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]

Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
#https://stats.stackexchange.com/questions/211436/why-normalize-images-by-subtracting-datasets-image-mean-instead-of-the-current
x_test = x_test - mean_image
"""In the ytrain it is preliminarily is in numbers just to indicate the particular class tf.keras.utils.to_categorical(y_train, num_classes=K) does is 
Converting the number to a binary vector
Ex: 
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)
tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)
"""
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))#shaping images into a vector Din=32x32x3
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
std=1e-5
w1 = std*np.random.randn(Din, K)#initialization
b1 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
batch_size = Ntr #for the batch size whole dataset is considered

iterations =100
lr =0.025
lr_decay=0.001#learning rate is decreased in each iteration
reg =0#regularization term which stops growth of weights
loss_history = []
train_acc_history = []
val_acc_history = []

seed = 0
rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    #indices = np.arange(Ntr)#generating image
    #rng.shuffle(indices)
    # Forward pass
    loss,cache=forward_prop(W=w1,b=b1,X=x_train,Y=y_train,reg=reg)
    y_hat=cache["y_hat"]
    # Backward pass
    dw1,db1=back_prop(Y_hat=y_hat,W=w1,X=x_train,Y=y_train,reg=reg)
    w1=w1-lr*dw1
    b1=b1-lr*db1

    test_loss,_=forward_prop(W=w1,b=b1,X=x_test,Y=y_test,reg=reg)
    print("Epoch "+str(t+1)+"/"+str(iterations)+">>> trainig_loss: ",loss,"testing_loss:",test_loss)
# Printing accuracies and displaying w as images


x_train: (50000, 32, 32, 3)
x_test: (10000, 32, 32, 3)
w1: (3072, 10)
b1: (10,)
Epoch 1/100>>> trainig_loss:  0.5000011469878922 testing_loss: 0.4411523414506603
Epoch 2/100>>> trainig_loss:  0.4421280447133633 testing_loss: 0.4290877051873943
Epoch 3/100>>> trainig_loss:  0.4293999247897458 testing_loss: 0.41166370237622524
Epoch 4/100>>> trainig_loss:  0.4120741842243922 testing_loss: 0.4016330864682733
Epoch 5/100>>> trainig_loss:  0.4019206746691254 testing_loss: 0.39192934300516447
Epoch 6/100>>> trainig_loss:  0.39220842705771747 testing_loss: 0.38409894245098813
Epoch 7/100>>> trainig_loss:  0.3843406723491497 testing_loss: 0.37696859938137334
Epoch 8/100>>> trainig_loss:  0.3771915260019078 testing_loss: 0.37062634515927473
Epoch 9/100>>> trainig_loss:  0.3708273918386447 testing_loss: 0.3648113337241597
Epoch 10/100>>> trainig_loss:  0.3649938711820334 testing_loss: 0.3594665793057578
Epoch 11/100>>> trainig_loss:  0.3596305826273817 testing_loss: 0.35450196612434576
Epoch 12/

In [21]:
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)
B=[[1,2,1,1],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
mse = np.mean((a - B)**2,axis=1)

print(mse)

def loss(W,b,X,Y,reg):
    print(W.shape,"W")
    print(b.shape,"b")
    print(X.shape,"X")
    print(Y.shape,"Y")
    prediction=np.dot(X,W)+np.transpose(b)
    print()
    m=X.shape[0]
    mse = np.sum(np.square((Y - prediction)**2))
    Loss=mse+ 1/(2*m)*reg*np.sum(W**2)
    return Loss

tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)
[1.5 0.  0.  0. ]


In [12]:
def sigmoid(z):
    z=np.array(z)
    return 1/(1+np.exp(-z))

sigmoid(B)

array([[0.73105858, 0.88079708, 0.73105858, 0.73105858],
       [0.5       , 0.73105858, 0.5       , 0.5       ],
       [0.5       , 0.5       , 0.73105858, 0.5       ],
       [0.5       , 0.5       , 0.5       , 0.73105858]])

In [1]:
def forward_prop(X,Y,b,W,reg):
    #print(W.shape,"W")
    #print(b.shape,"b")
    #print(X.shape,"X")
    #print(Y.shape,"Y")
    y_hat=np.dot(X,W)+np.transpose(b)
    #print(A.shape,"A")
    m=X.shape[0]
    #print(m)
    mse =1/(2*m)* np.sum(np.square((Y - y_hat)**2))
    Loss=mse+ 1/(2*m)*reg*np.sum(W**2)
    cache={"y_hat":y_hat}
    return Loss,cache

In [4]:
loss,cache=forward_prop(W=w1,b=b1,X=x_train,Y=y_train,reg=reg)
print(loss)
y_hat=cache["y_hat"]

0.49998923857004474


In [9]:

def back_prop(W,X,Y,Y_hat,reg):
    m=Y.shape[0]
    dy_hat =Y_hat-Y
    #print(dy_hat.shape,"dy_hat")
    #print(dA.shape,"dA")
    dW =(1/float(m))*(np.dot(X.T, dy_hat)+reg*W)
    db=(1/float(m))*np.sum(dy_hat,axis=0)
    #print(dW.shape)
    #print(db.shape)
    return dW,db

In [10]:
back_prop(Y_hat=y_hat,W=w1,X=x_train,Y=y_train,reg=reg)

(array([[-5.60294253e-03, -2.38228143e-03,  2.25626789e-03, ...,
         -1.66584013e-03, -6.09623615e-03, -1.43466064e-02],
        [-9.87432428e-03, -8.76206678e-04,  1.98903826e-03, ...,
         -2.10664917e-03, -1.04874780e-02, -1.61687181e-02],
        [-1.79099455e-02, -1.37219492e-03,  6.71838721e-03, ...,
         -1.92025827e-03, -1.81712554e-02, -2.04092694e-02],
        ...,
        [-1.25619369e-03, -2.77595903e-03,  5.82931016e-04, ...,
         -5.32556245e-03,  1.18562262e-02, -5.05095503e-03],
        [-3.97208294e-03, -2.21981253e-03, -4.58652880e-05, ...,
         -3.55784550e-03,  6.31214200e-03, -3.90185596e-03],
        [-9.14413372e-03, -4.84109105e-03,  2.94591984e-03, ...,
          2.34340162e-03, -1.38188891e-03, -5.50754172e-03]]),
 array([-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]))